# Entrenamiento de YOLOv8 en Google Colab

### 1. Montar Google Drive e Instalar Librerías

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install ultralytics

### 2. Configuración y Descompresión del Dataset

In [ ]:
import os
import tarfile
from ultralytics import YOLO

# --- PARÁMETROS DE CONFIGURACIÓN ---
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/Colab Notebooks/mod03-projects/project-xii-computer-vision'

# --- Dataset (Comprimido) ---
COMPRESSED_DATASET_NAME = 'dataset_v2_yolov8_obb.tar.gz'
COMPRESSED_DATASET_PATH = os.path.join(DRIVE_PROJECT_PATH, COMPRESSED_DATASET_NAME)
DATASET_BASE_PATH = '/content/dataset_v2_yolov8_obb'
DATASET_YAML_PATH = os.path.join(DATASET_BASE_PATH, 'data.yaml')

# --- Lógica de Descompresión ---
if not os.path.exists(DATASET_BASE_PATH):
    print(f"Dataset no encontrado en {DATASET_BASE_PATH}. Descomprimiendo...")
    if not os.path.exists(COMPRESSED_DATASET_PATH):
        raise FileNotFoundError(f"El archivo {COMPRESSED_DATASET_PATH} no existe.")
    with tarfile.open(COMPRESSED_DATASET_PATH, 'r:gz') as tar_ref:
        tar_ref.extractall('/content/')
    print("Descompresión completada.")
else:
    print(f"Dataset ya encontrado en {DATASET_BASE_PATH}.")

# --- Configuración del Modelo y Entrenamiento ---
MODEL_NAME = 'yolov8s.pt'  # CORRECTO: Usando el modelo Small
EPOCHS = 80  # Mantenemos 80 épocas como solicitaste
# CAMBIO: Nuevo nombre para no sobreescribir resultados anteriores
RUN_NAME = 'yolov8s_obb_b16_w4_img640_ep80_colab_run_5'
BATCH_SIZE = 12 # CAMBIO: Aumentado a 16 para acelerar, es seguro con el modelo 's'
IMG_SIZE = 640 # NOTA: Bajar a 416 aceleraría aún más el entrenamiento si fuera necesario
WORKERS = 4
PATIENCE = 20
CACHE = False
OPTIMIZER = 'AdamW'
COS_LR = True

print('--- CONFIGURACIÓN DE ENTRENAMIENTO ---')
print(f'Modelo: {MODEL_NAME}')
print(f'Epochs: {EPOCHS}')
print(f'Batch Size: {BATCH_SIZE}')
print(f'Image Size: {IMG_SIZE}')
print('----------------------------------------')
print('--- COMPROBACIONES DE DATASET, YAML Y MODELO ---')
print(f"Dataset listo para usar en: {DATASET_BASE_PATH}")
print(f"Archivo YAML: {DATASET_YAML_PATH}")
print(f"Modelo base: {MODEL_NAME}")
print('----------------------------------------')

model = YOLO(MODEL_NAME)
results = model.train(
    data=DATASET_YAML_PATH,
    project=os.path.join(DRIVE_PROJECT_PATH, 'training_results'),
    name=RUN_NAME,
    epochs=EPOCHS,
    patience=PATIENCE,
    batch=BATCH_SIZE,
    imgsz=IMG_SIZE,
    workers=WORKERS,
    cache=CACHE,
    optimizer=OPTIMIZER,
    cos_lr=COS_LR,
    device='0',
    plots=True,
    save_json=True
)

print('--- ENTRENAMIENTO FINALIZADO ---')
print(f'Resultados guardados en: {results.save_dir}')